In [1]:
import os, sys
import numpy as np


from lace.emulator.nn_architecture import MDNemulator_polyfit
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator
from lace.emulator.emulator import P1D_emulator


/data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d
from lace.archive import pnd_archive
from lace.archive import interface_archive

In [3]:
emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']

# CREATE TRAINING AND TESTING ARCHIVE

In [4]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

In [5]:
archive_test = pnd_archive.archivePND(sim_suite="Cabayol23", pick_sim='central')
archive_test.get_testing_data()
len(archive_test.testing_data)

11

In [6]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:43]

# CREATE EMULATOR OBJECT

### The class default arguments are:

- archive=None archive with the training data
- emulator_label=None Pre-defined emulator's labels (Pedersen21, Pedersen23, Cabayol23)
- emu_algorithm=None Emulator algorithm (NN, GP)
- sims_label=None Simulations suite to use (Gadget, Nyx)
- zmax=4.5 Max redshfit of the training sample
- kmax_Mpc=4 Max scale of the emulator
- ndeg=5 Polynomial order of the polynomial fit if emu_type =='polyfit'
- train=True (whether to train or not the NN emulator. If train is False, a model_path must be provided
- emu_type="polyfit" (k_bin, polyfit)
- model_path=None (path to pretrained NN emulator)
- save_path=None (path to save the trained NN emulator)
- nepochs_nn=100  (Number of training iterations of the NN emulator) 

### If an emulator label is provided, the interface selects the emulator parameters and  archive from that emulator option

### If the emulator label is not provided, an archive, a emu_algorithm and a sims_label must be given

## GP emulators from Pedersen+21 and Pedersen+23

In [15]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:33]

In [11]:
emulator = P1D_emulator(emulator_label='Pedersen21')

Selected pre-tuned emulator
Select emulator used in Pedersen et al. 2021
Gaussian Process emulator predicting the P1D at each k-bin. It goes to scales of 3Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones.
Training GP on 330 points
GPs optimised in 2.71 seconds


In [16]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

–-----------------------------------

In [20]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:43]

In [18]:
emulator = P1D_emulator(emulator_label='Pedersen23')

Selected pre-tuned emulator
Select emulator used in Pedersen et al. 2023
Gaussian Process emulator predicting the optimal P1Dfitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones
Training GP on 330 points
GPs optimised in 0.93 seconds


In [21]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

3.909537524467298
2.978695256736989
Warning! Your requested k bins are higher than the training values.


### NN emulator from Cabayol-Garcia+23

In [7]:
emulator = P1D_emulator(emulator_label='Cabayol23')

Selected pre-tuned emulator
Select emulator used in Cabayol-Garcia et al. 2023
Neural network emulator predicting the optimal P1D fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones
Train emulator
Selected pre-tuned training set
start the training of the emulator
Training network on 9900
Emualtor trained in 0.9109842777252197 seconds


In [8]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc, return_covar=True)

### Custom emulators

In [9]:
emulator = P1D_emulator(archive, emu_algorithm='GP',sims_label='Gadget')

Selected custome emulator
Training GP on 330 points
GPs optimised in 1.07 seconds


In [7]:
emulator = P1D_emulator(archive, emu_algorithm='NN',sims_label='Gadget')

Selected custome emulator
Selected custome training set
Loading emulator using a specific archive
start the training of the emulator
Training network on 330
Emualtor trained in 3.018554449081421 seconds


In [8]:
emulator = P1D_emulator(archive, emu_algorithm='NN',sims_label='Nyx')

Selected custome emulator


Exception: Work in progress

## The neural network emulator supposrts running both on GPU and CPU. No argument is needed, f the gpu is available, it will use it. Otherwise, the emulator trains on CPUs

## The neural network emulator uses the following default parameters:

- paramList=['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
- nepochs=100
- step_size=75
- kmax_Mpc=4
- ndeg=5. For the extended version (kmax_Mpc=8), set to 7
- initial_weights=True. Always starts from the same iniial parameters. These were selected randomly. Set to False if random initialization is required.

## Additioanlly, we must provide either an archive 

In [7]:
emulator = NNEmulator(archive=archive, nepochs=1)

Selected custome training set
Loading emulator using a specific archive
start the training of the emulator
Training network on 9900
Emualtor trained in 0.9789836406707764 seconds


## or a training_set label

In [8]:
emulator = NNEmulator(training_set='Cabayol23', nepochs=1)

TypeError: NNEmulator.__init__() got an unexpected keyword argument 'sim_suite'

## We can also load a pre-trained emulator, providing the path of the saved network parameters:

In [12]:
emulator = NNEmulator(archive=archive,nepochs=1, model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)

Selected custome training set
Loading emulator using a specific archive
Model loaded. No training needed


### Please ensure that the emulator parameters used during training are the same as the ones you are using for testing when loading a pre-trained emulator.


## Once the emulator is trained, to emulate the p1d given some model parameters:

In [15]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

## The Gaussian process emulator uses the following default parameters:

- paramList=['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
- kmax_Mpc=10
- ndeg=4

In [20]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

## We must also provide either a training_set label

In [21]:
emulator = GPEmulator(training_set='Pedersen21')

 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 1.44 seconds


## or an archive

In [22]:
emulator = GPEmulator(archive=archive)

Training GP on 330 points
GPs optimised in 1.09 seconds


## Once the emulator is trained, to emulate the p1d given some model parameters:

In [ ]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

In [9]:
emulator = GPEmulator(training_set='Pedersen21')

 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 0.82 seconds


In [10]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

## LOAD TRAINED NEURAL NETWORK EMULATOR 

In [7]:
emulator = P1D_emulator(archive, emu_algorithm='NN',sims_label='Gadget',model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)

Selected custome emulator
Selected custome training set
Loading emulator using a specific archive
Model loaded. No training needed


In [8]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

### train the emulator

In [ ]:
emulator = P1D_emulator(emulator_label='Cabayol23',nepochs_nn=1)

### Load trained emulator

In [ ]:
emulator = P1D_emulator(emulator_label='Cabayol23', train=False, model_path='NNmodels/NNEmulator_LaCEHC.pt')

In [ ]:
# This is supposed to fail
emulator = P1D_emulator()

In [ ]:
emulator = P1D_emulator(archive=archive, emu_algorithm='NN')

In [ ]:
emulator = P1D_emulator(archive=archive, emu_algorithm='GP')